# Read Data

In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)


In [3]:
import pandas as pd 
fp = pd.ExcelFile('..executive_ai_dashboard/data/linkedin.xlsx')

# Get the list of sheet names
sheet_names = fp.sheet_names

# Read each sheet into a DataFrame
data = {}
for sheet_name in sheet_names:
    data[sheet_name] = pd.read_excel(fp, sheet_name, engine='openpyxl')

FileNotFoundError: [Errno 2] No such file or directory: '..executive_ai_dashboard/data/linkedin.xlsx'

# Pre-processing

In [ ]:
engagements = data['ENGAGEMENT']
engagements['Date'] = pd.to_datetime(engagements['Date'])
engagements.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         365 non-null    datetime64[ns]
 1   Impressions  365 non-null    int64         
 2   Engagements  365 non-null    int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 8.7 KB


In [ ]:
engagements['DayOfWeek'] = engagements['Date'].dt.day_name()
engagements['EngagementRate'] = (engagements['Engagements'] / engagements['Impressions']) * 100
engagements['EngagementRate'] = engagements['EngagementRate'].map("{:.2f}".format)
engagements['EngagementRate'] = engagements['EngagementRate'].astype(float)
engagements.head()

,Date,Impressions,Engagements,DayOfWeek,EngagementRate
0,2023-05-02,1890,40,Tuesday,2.12
1,2023-05-03,1887,46,Wednesday,2.44
2,2023-05-04,1935,163,Thursday,8.42
3,2023-05-05,3229,84,Friday,2.60
4,2023-05-06,1497,59,Saturday,3.94


In [ ]:
engagements_by_day = engagements.groupby('DayOfWeek')[['Engagements','Impressions']].sum().reset_index()
engagements_by_day['EngagementRate'] = (engagements_by_day['Engagements'] / engagements_by_day['Impressions']) * 100
engagements_by_day['EngagementRate'] = engagements_by_day['EngagementRate'].map("{:.2f}".format)
engagements_by_day = engagements_by_day.sort_values(by='Engagements', ascending=False).reset_index(drop=True)
engagements_by_day


,DayOfWeek,Engagements,Impressions,EngagementRate
0,Monday,3161,95976,3.29
1,Friday,2109,78376,2.69
2,Thursday,1804,81530,2.21
3,Wednesday,1782,83279,2.14
4,Tuesday,1637,88792,1.84
5,Sunday,1510,51921,2.91
6,Saturday,1303,52528,2.48


In [ ]:
demographics = data['DEMOGRAPHICS']

demographics = demographics[demographics['Percentage'] != '< 1%'].copy()
demographics.loc[:, 'Percentage'] = pd.to_numeric(demographics['Percentage'], errors='coerce')
demographics


,Top Demographics,Value,Percentage
0,Job titles,Application Engineer,0.034703
1,Job titles,Software Engineer,0.026637
2,Job titles,Founder,0.022885
3,Job titles,Chief Executive Officer,0.02176
4,Job titles,Account Executive,0.017258
5,Locations,San Francisco Bay Area,0.26768
6,Locations,"Austin, Texas Metropolitan Area",0.063403
7,Locations,Dallas-Fort Worth Metroplex,0.05815
8,Locations,Greater Bengaluru Area,0.046708
9,Locations,New York City Metropolitan Area,0.032639


In [ ]:
followers = data['FOLLOWERS']
# Extract the total followers value
total_followers = followers.columns[1]
# Make the third row as the column titles
new_header = followers.iloc[1]
followers.columns = new_header
# Drop the first two rows
followers = followers.drop([0, 1])
# Reset the index
followers = followers.reset_index(drop=True)
followers['Date'] = pd.to_datetime(followers['Date'])
followers.head()

1,Date,New followers
0,2023-05-02,4
1,2023-05-03,9
2,2023-05-04,5
3,2023-05-05,6
4,2023-05-06,5


In [ ]:
df = followers.copy()
df['YearMonth'] = df['Date'].dt.to_period('M')
df['YearMonth'] = df['YearMonth'].astype(str)
monthly_followers = df.groupby('YearMonth')['New followers'].sum().reset_index()
import plotly.express as px

fig = px.line(monthly_followers, x='YearMonth', y='New followers', title='New Followers Over Time')
fig.show()


In [ ]:
top_posts = data['TOP POSTS']
new_header = top_posts.iloc[1]
top_posts.columns = new_header
top_posts = top_posts.drop([0,1])
top_posts = top_posts.dropna(axis=1, how='all')
top_posts.head()

1,Post URL,Post publish date,Engagements,Post URL,Post publish date,Impressions
2,https://www.linkedin.com/feed/update/urn:li:ac...,7/17/2023,835,https://www.linkedin.com/feed/update/urn:li:ac...,8/9/2023,57830
3,https://www.linkedin.com/feed/update/urn:li:ac...,8/9/2023,530,https://www.linkedin.com/feed/update/urn:li:ac...,6/26/2023,18304
4,https://www.linkedin.com/feed/update/urn:li:ac...,12/24/2023,511,https://www.linkedin.com/feed/update/urn:li:ac...,1/17/2024,16195
5,https://www.linkedin.com/feed/update/urn:li:ac...,6/1/2023,405,https://www.linkedin.com/feed/update/urn:li:ac...,2/5/2024,12187
6,https://www.linkedin.com/feed/update/urn:li:ac...,11/23/2023,337,https://www.linkedin.com/feed/update/urn:li:ac...,9/11/2023,10788


In [ ]:
# split impressions and engageents data
top_posts_engagements = top_posts.iloc[:,:3]
top_posts_impressions = top_posts.iloc[:,3:]

merged_data = pd.merge(top_posts_engagements, top_posts_impressions, on='Post URL', how='inner')
merged_data = merged_data.rename(columns={'Engagements_x':'Engagements','Engagements_y':'Impressions','Post publish date_x':'Post publish date'})
merged_data = merged_data.drop(columns=['Post publish date_y'])
merged_data.head()

1,Post URL,Post publish date,Engagements,Impressions
0,https://www.linkedin.com/feed/update/urn:li:ac...,7/17/2023,835,7548
1,https://www.linkedin.com/feed/update/urn:li:ac...,8/9/2023,530,57830
2,https://www.linkedin.com/feed/update/urn:li:ac...,12/24/2023,511,10631
3,https://www.linkedin.com/feed/update/urn:li:ac...,6/1/2023,405,8802
4,https://www.linkedin.com/feed/update/urn:li:ac...,11/23/2023,337,10124


In [11]:
top_posts_preview = pd.read_csv('data/top_posts_with_topics.csv')
top_posts_preview['Post publish date'] = pd.to_datetime(top_posts_preview['Post publish date'])
top_posts_preview.head()

,Unnamed: 0,Post URL,Post publish date,Engagements,Impressions,Title,Description,Thumbnail,Topics
0,0,https://www.linkedin.com/feed/update/urn:li:ac...,2023-07-17,835,7548,KT Moore on LinkedIn: #honoredandgrateful #dei...,I want to express my heartfelt thanks to SEMI ...,https://media.licdn.com/dms/image/D5610AQGkQ-Y...,"DEIB Recognition, Gratitude, SEMI Honor, Shapi..."
1,1,https://www.linkedin.com/feed/update/urn:li:ac...,2023-08-09,530,57830,KT Moore on LinkedIn: How Artificial Intellige...,Behind every Formula One (#F1) car tearing up ...,https://media.licdn.com/dms/image/sync/D5627AQ...,"Engineering Excellence, Team Collaboration, Hi..."
2,2,https://www.linkedin.com/feed/update/urn:li:ac...,2023-12-24,511,10631,KT Moore on LinkedIn: #holidayseason #happyhol...,"As the year comes to a close, let&#39;s expres...",https://media.licdn.com/dms/image/D5622AQGAiid...,"Gratitude, Reflection, Growth, Appreciation, N..."
3,3,https://www.linkedin.com/feed/update/urn:li:ac...,2023-06-01,405,8802,KT Moore on LinkedIn: #innovation #intelligent...,Excited to be part of this morning’s opening b...,https://media.licdn.com/dms/image/D5622AQEhpM_...,"Nasdaq Ceremony, CEO Devgan, Trading, Cadence ..."
4,4,https://www.linkedin.com/feed/update/urn:li:ac...,2023-11-23,337,10124,KT Moore on LinkedIn: #thanksgiving2023 #thank...,"This Thanksgiving, I encourage you to not only...",https://media.licdn.com/dms/image/D5610AQEH9et...,"Gratitude, Community, Compassion, Inclusivity,..."


In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


def get_post_info(url):
    try:
        # Fetch the webpage
        response = requests.get(url)
        response.raise_for_status()

        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find relevant Open Graph tags
        og_title = soup.find('meta', property='og:title')
        og_description = soup.find('meta', property='og:description')
        og_image = soup.find('meta', property='og:image')

        # Extract the content attribute which contains the information
        title = og_title['content'] if og_title else "Title not found"
        description = og_description['content'] if og_description else "Description not found"
        thumbnail_url = og_image['content'] if og_image else None

        return title, description, thumbnail_url
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None, None
    

urls = merged_data['Post URL']

# Initialize lists to store extracted information
titles = []
descriptions = []
thumbnails = []
for url in urls:
    title, description, thumbnail_url = get_post_info(url)
    titles.append(title)
    descriptions.append(description)
    thumbnails.append(thumbnail_url)
# Create a DataFrame to store the data
data_post_preview = {
    'Title': titles,
    'Description': descriptions,
    'Thumbnail': thumbnails
}


In [12]:
data_post_preview = pd.DataFrame(data_post_preview)
top_posts_preview = pd.concat([merged_data, data_post_preview], axis=1)
top_posts_preview.head()

,Post URL,Post publish date,Engagements,Impressions,Title,Description,Thumbnail
0,https://www.linkedin.com/feed/update/urn:li:ac...,7/17/2023,835,7548,KT Moore on LinkedIn: #honoredandgrateful #dei...,I want to express my heartfelt thanks to SEMI ...,https://media.licdn.com/dms/image/D5610AQGkQ-Y...
1,https://www.linkedin.com/feed/update/urn:li:ac...,8/9/2023,530,57830,KT Moore on LinkedIn: How Artificial Intellige...,Behind every Formula One (#F1) car tearing up ...,https://media.licdn.com/dms/image/sync/D5627AQ...
2,https://www.linkedin.com/feed/update/urn:li:ac...,12/24/2023,511,10631,KT Moore on LinkedIn: #holidayseason #happyhol...,"As the year comes to a close, let&#39;s expres...",https://media.licdn.com/dms/image/D5622AQGAiid...
3,https://www.linkedin.com/feed/update/urn:li:ac...,6/1/2023,405,8802,KT Moore on LinkedIn: #innovation #intelligent...,Excited to be part of this morning’s opening b...,https://media.licdn.com/dms/image/D5622AQEhpM_...
4,https://www.linkedin.com/feed/update/urn:li:ac...,11/23/2023,337,10124,KT Moore on LinkedIn: #thanksgiving2023 #thank...,"This Thanksgiving, I encourage you to not only...",https://media.licdn.com/dms/image/D5610AQEH9et...


### Topic Modelling

In [53]:
from bertopic import BERTopic

df = top_posts_preview.copy()
docs = df.Description.to_list()
model = BERTopic.load("davanstrien/chat_topics")
topics, probabilities = model.transform(docs)

model.visualize_barchart()


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

2024-05-11 11:05:43,027 - BERTopic - Predicting topic assignments through cosine similarity of topic and document embeddings.


In [71]:
df.Description[1]

'Behind every Formula One (#F1) car tearing up the circuit at 250 mph is a team of engineers and scientists competing to wrangle every advantage, leveraging the… | 15 comments on LinkedIn'

In [79]:
from openai import OpenAI

client = OpenAI()

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role":"system", "content":'You are a topic modelling system',},
                {"role":"user", "content": prompt}]
    response = client.chat.completions.create(model=model,messages=messages, temperature=0, )
    return response.choices[0].message.content

for index, row in top_posts_preview.iterrows():
    prompt = f"""
    Suggest up to five single or two-word topics for a social media post related to {row['Description']}. Print them in one line separted by a comma.
    """
    response = get_completion(prompt)

    # Assuming you want to update a new column named 'Topics' in the DataFrame
    top_posts_preview.at[index, 'Topics'] = response

top_posts_preview.head()

,Post URL,Post publish date,Engagements,Impressions,Title,Description,Thumbnail,Topics
0,https://www.linkedin.com/feed/update/urn:li:ac...,7/17/2023,835,7548,KT Moore on LinkedIn: #honoredandgrateful #dei...,I want to express my heartfelt thanks to SEMI ...,https://media.licdn.com/dms/image/D5610AQGkQ-Y...,"DEIB Recognition, Gratitude, SEMI Honor, Shapi..."
1,https://www.linkedin.com/feed/update/urn:li:ac...,8/9/2023,530,57830,KT Moore on LinkedIn: How Artificial Intellige...,Behind every Formula One (#F1) car tearing up ...,https://media.licdn.com/dms/image/sync/D5627AQ...,"Engineering Excellence, Team Collaboration, Hi..."
2,https://www.linkedin.com/feed/update/urn:li:ac...,12/24/2023,511,10631,KT Moore on LinkedIn: #holidayseason #happyhol...,"As the year comes to a close, let&#39;s expres...",https://media.licdn.com/dms/image/D5622AQGAiid...,"Gratitude, Reflection, Growth, Appreciation, N..."
3,https://www.linkedin.com/feed/update/urn:li:ac...,6/1/2023,405,8802,KT Moore on LinkedIn: #innovation #intelligent...,Excited to be part of this morning’s opening b...,https://media.licdn.com/dms/image/D5622AQEhpM_...,"Nasdaq Ceremony, CEO Devgan, Trading, Cadence ..."
4,https://www.linkedin.com/feed/update/urn:li:ac...,11/23/2023,337,10124,KT Moore on LinkedIn: #thanksgiving2023 #thank...,"This Thanksgiving, I encourage you to not only...",https://media.licdn.com/dms/image/D5610AQEH9et...,"Gratitude, Community, Compassion, Inclusivity,..."


# Dash

In [12]:
from dash import Dash, html, dcc, callback_context
import plotly.express as px
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from datetime import timedelta

# Assuming 'data' is defined and contains the required DataFrame


total_engagements = engagements['Engagements'].sum()
total_impressions = engagements['Impressions'].sum()
engagement_rate = (total_engagements/total_impressions)*100
emv = (engagement_rate*total_impressions*6.59)/100
optimal_day = engagements_by_day.loc[0, 'DayOfWeek']
data = top_posts_preview.sort_values(by='Engagements', ascending=False).reset_index(drop=True)

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = 'LinkedIn Analytics'


app.layout = html.Div([
    html.Img(src='./assets/logo1.png',style={'width':'100px', 'margin-top': '20px', 'margin-left': '100px', 'margin-right': '100px'}),
    html.H1("LinkedIn Analytics Dashboard", style={'text-align': 'center', 'color': 'black', 'font-family': 'Arial, sans-serif'}),
    dcc.Markdown('**Choose a date range:** ', style={'margin-top': '20px', 'margin-left': '100px', 'margin-right': '100px',}),
    dcc.DatePickerRange(
        id='date-picker-range',
        min_date_allowed=engagements['Date'].min(),
        max_date_allowed=engagements['Date'].max(),
        initial_visible_month=engagements['Date'].max(),
        start_date=engagements['Date'].min(),
        end_date=engagements['Date'].max(),
        style={'margin-top': '0px', 'margin-left': '100px', 'margin-right': '100px',}),
    dbc.Row([ 
        dcc.Markdown('**Overview** ', style={'margin-top': '20px', 'margin-left': '100px', 'margin-right': '100px', 'text-decoration': 'underline'}),
        html.Div([
            html.Div(html.Img(src='./assets/impression.png', style={'width': '50px', 'alignSelf': 'left','margin-right': '0px'})),
            html.Div(id='impressions', children=f'Impressions: {total_impressions}', style={ 'text-align': 'center', 'margin-top': '20px', 'color': 'black', 'font-family':'Helvetica Black'})
            ], style={'display': 'flex', 'align-items': 'center', 'justify-content': 'center', 'width': '200px', 'height': '100px', 
                  'background-color': 'white', 'margin-top': '0px', 'border-radius': '10px','margin-right': '20px', 'margin-left':'100px',  'border': '1px solid #ccc', 'box-shadow': '2px 2px 5px #888888'}),
        html.Div([
            html.Div(html.Img(src='./assets/engagement-icon.png', style={'width': '50px', 'alignSelf': 'left','margin-right': '0px'})),
            html.Div(id='engagements', children=f'Engagements: {total_engagements}', style={ 'text-align': 'center', 'margin-top': '20px', 'color': 'black', 'font-family':'Helvetica Black'})
            ], style={'display': 'flex', 'align-items': 'center', 'justify-content': 'center', 'width': '200px', 'height': '100px', 
                  'background-color': 'white', 'margin-top': '0px', 'border-radius': '10px','margin-right': '20px', 'margin-left':'100px',  'border': '1px solid #ccc', 'box-shadow': '2px 2px 5px #888888'}),
        html.Div([
            html.Div(html.Img(src='./assets/engagement.png', style={'width': '50px', 'alignSelf': 'left'})),
            html.Div(id='engagement_rate', children=f'Engagement Rate: {engagement_rate:.2f}%', style={'text-align': 'center', 'margin-left': '20px', 'color': 'black','font-family': 'Helvetica Black'})
            ], style={'display': 'flex', 'align-items': 'center', 'justify-content': 'center', 'width': '200px', 'height': '100px', 
                  'background-color': 'white', 'margin-top': '0px', 'border-radius': '10px', 'margin-right': '20px', 'margin-left':'20px',  'border': '1px solid #ccc', 'box-shadow': '2px 2px 5px #888888'}),

        html.Div([
            html.Div(html.Img(src='./assets/like.png', style={'width': '50px', 'alignSelf': 'left'})),
            html.Div(id='emv_value', children=f'Earned Media Value: {emv:.2f}%', style={'text-align': 'center', 'margin-left': '20px', 'color': 'black', 'font-family': 'Helvetica Black'})
            ], style={'display': 'flex', 'align-items': 'center', 'justify-content': 'center', 'width': '200px', 'height': '100px', 
                  'background-color': 'white', 'margin-top': '0px', 'border-radius': '10px', 'margin-right': '20px', 'margin-left':'20px',  'border': '1px solid #ccc', 'box-shadow': '2px 2px 5px #888888'}),
        
        html.Div([
            html.Div(html.Img(src='./assets/followers.png', style={'width': '50px', 'alignSelf': 'left', 'margin-right': '50px'})),
            html.Div(id='followers', children=['Followers', html.Br(), html.B("{:,.0f}".format(total_followers), style={'text-align': 'center', 'margin-left': '20px','color':'black', 'font-size':'20px', 'margin-top':'40px', 'font-family':'Arial Black' })])
            ], style={'display': 'flex', 'align-items': 'center', 'justify-content': 'center', 'width': '200px', 'height': '100px', 
                  'background-color': 'white', 'margin-top': '0px', 'border-radius': '10px', 'margin-right': '20px', 'margin-left':'20px',  'border': '1px solid #ccc', 'box-shadow': '2px 2px 5px #888888'}),
    ]),

    dbc.Row([
        dcc.Markdown('**Follower Growth Trend** ', style={'margin-top': '40px', 'margin-left': '100px', 'margin-right': '100px', 'text-decoration': 'underline'}),
        html.P('Follower Growth shows the increase in the number of your followers. ', style={'margin-left': '100px', 'margin-right': '100px'}),

        dcc.Graph(
            figure=px.line(monthly_followers, x='YearMonth', y='New followers', color_discrete_sequence=['#b51a00'],  markers=True),
            style= {'width':'800px', 'height':'500px','margin-top': '0px', 'margin-right': '20px', 'Margin-left':'100px'},
        ),

        dbc.Row([
            dcc.Dropdown(
                id='audience-dropdown',
                options=[
                    {'label': 'Job titles', 'value': 'Job titles'},
                    {'label': 'Locations', 'value': 'Locations'},
                    {'label': 'Industries', 'value': 'Industries'},
                    {'label': 'Seniority', 'value': 'Seniority'},
                    {'label': 'Company size', 'value': 'Company size'},
                    {'label': 'Companies', 'value': 'Companies'},
                ],
                value='Job titles',  
                style={'margin-top': '0px', 'width': '200px', 'margin-left': '40px', 'margin-right': '100px'}), 

            dcc.Graph(
                id='audience-graph',
                figure = px.bar(demographics[demographics['Top Demographics'] == 'Industries'], x='Percentage', y='Value', title=f'Top Industries by Percentage', labels={'Percentage': 'Percentage (%)', 'Value': 'Top Values'}).update_layout(plot_bgcolor='white'),
                style= {'height':'400px','margin-top': '0px', 'margin-right': '20px', 'Margin-left':'100px',})
                ], style= {'width':'500px','height':'400px','margin-top': '0px', 'margin-right': '20px', 'Margin-left':'10px'}),
    ]),

    dbc.Row([
        dcc.Markdown('**Reach/Reactions Trend** ', style={'margin-top': '40px', 'margin-left': '100px', 'margin-right': '100px', 'text-decoration': 'underline'}),
        html.P('Follower Growth shows the increase in the number of your followers. ', style={'margin-left': '100px', 'margin-right': '100px'}),
        dcc.Dropdown(
            id='variable-dropdown',
            options=[
                {'label': 'Engagements', 'value': 'Engagements'},
                {'label': 'Impressions', 'value': 'Impressions'},
                {'label': 'Engagement Rate', 'value': 'EngagementRate'},
            ],
            value='Engagements',
            style={'margin-top': '0px', 'width':'200px','margin-left': '40px', 'margin-right': '100px'}),

        dbc.Row([    
            dcc.Graph(
                id='engagements-graph',
                figure=px.line(engagements, x='Date', y='Impressions', color_discrete_sequence=['#b51a00']),
                style={'width':'1000px','height':'500px','margin-top': '0px', 'margin-right': '0px', 'Margin-left':'10px',}),
            
            dbc.Row([
                dcc.Graph(
                    id='optimal-graph',
                    figure=px.histogram(engagements_by_day, x='DayOfWeek', y='Engagements'),
                    style={'width':'500px','height':'250px','margin-top': '0px', 'margin-right': '20px', 'Margin-left':'0px'}),

                html.Div([ 
                    html.B('What is the best day of week to post? \n', style={'margin-top': '0px', 'margin-right': '20px', 'Margin-left':'100px','text-align': 'center', 'color': 'gray', 'font-family': 'Helvetica Neue'}),
                    html.B(id ='best-day', children='Reach or Engagement', style={'margin-top': '20px', 'margin-right': '10px', 'font-size':'20px','Margin-left':'10px','align-self': 'center', 'color': 'black', 'font-family': 'Helvetica Neue'}),
                    html.P(id ='optimal-days',children=["Optimal days:", html.B(optimal_day, style={'font-size':'1000px'})], style={'margin-top': '10px', 'margin-right': '20px', 'Margin-left':'10px', 'color': 'Black', 'font-family': 'Arial, sans-serif'})],
                    style={'width':'400px','height':'200px','margin-top': '0px', 'margin-right': '20px', 'Margin-left':'10px'})
            ], style = {'width':'500px',})
        ]),  
    ]),


    dcc.Dropdown(
        id='posts-dropdown',
        options=[
            {'label': 'Top engaging posts', 'value': 'Top engaging posts'},
            {'label': 'Top posts by reach', 'value': 'Top posts by reach'},
        ],
        value='Top engaging posts',  
        style={'margin-top': '20px', 'width': '400px', 'margin-left': '40px', 'margin-right': '100px'}),


    dbc.Table(id = 'top-posts-preview',
        children=[
            html.Tr([
                html.Th("Published on"),
                html.Th("Posts"),
                html.Th(""),
                html.Th("Impressions"),
                html.Th("Reactions"),
                html.Th("Post Link"),
                html.Th("Topics"),
            ], style={'background-color': 'lightgray'}),  # Header row styling
        ] +
        # Rows
        [html.Tr([
            html.Td(row['Post publish date'], style={'width':'200px', 'margin-left': '10px'}),
            html.Td(html.Img(src=row['Thumbnail'], height=100) if row['Thumbnail'] else html.P('Thumbnail not found.')),
            html.Td(row['Description'],style={'width':'800px', 'margin-left': '10px'}),
            html.Td(row['Impressions'], style={'width':'200px', 'margin-left': '10px'}),
            html.Td(row['Engagements'],style={'width':'200px',}),
            html.Td(html.A("View on LI", href=row['Post URL']), style={'width':'200px',}),
            html.Td(row['Topics'],style={'width':'300px',}),
        ], style={'border-bottom': '1px solid gray', 'padding': '10px','width':'1700px'}) for _, row in data.iterrows()],
        bordered=True,
        responsive=True,
        striped=True,
        hover=True,
        style={'margin': '20px', 'width': '100%'}  # Overall table styling
    ),  

], style = {'width':'1800px'})

def calculate_percentage_change(current_value, previous_value):
    if previous_value != 0:
        return round(((current_value - previous_value) / previous_value) * 100, 2)
    else:
        return 0
@app.callback(
    Output('engagements', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_engagements(start_date, end_date):
    filtered_engagements = engagements[(engagements['Date'] >= start_date) & (engagements['Date'] <= end_date)]
    total_engagements = filtered_engagements['Engagements'].sum()
    # Calculate previous period start and end dates based on selected period
    selected_period_start = pd.to_datetime(start_date)
    selected_period_end = pd.to_datetime(end_date)
    previous_period_start = selected_period_start - (selected_period_end - selected_period_start)
    previous_period_end = selected_period_start - timedelta(days=1)  # End 1 day before the selected period start
    
    # Filter engagements for previous period
    previous_period_engagements = engagements[(engagements['Date'] >= previous_period_start) & (engagements['Date'] <= previous_period_end)]
    previous_period_total_engagements = previous_period_engagements['Engagements'].sum()
    
    percentage_change = calculate_percentage_change(total_engagements, previous_period_total_engagements)
    if percentage_change < 0:
        return ['Engagements ',html.Br(), html.B("{:,.0f}".format(total_engagements), style= {'color':'black', 'font-size':'20px', 'margin-top':'40px', 'font-family':'Arial Black' }), html.Br(),  html.P(f"{percentage_change}%", style= {'width':'100px','display': 'flex', 'align-items': 'center', 'justify-content': 'center',
                  'background-color': 'white', 'margin-top': '20px', 'border-radius': '10px', 'margin-right': '20px', 'margin-left':'20px', 'font-size':'12px', 'color':'red'}) ]
    elif percentage_change > 0:
        return ['Engagements ',html.Br(), html.B("{:,.0f}".format(total_engagements), style= {'color':'black', 'font-size':'20px', 'margin-top':'40px', 'font-family':'Arial Black' }), html.Br(),  html.P(f"{percentage_change}%", style= {'width':'100px','display': 'flex', 'align-items': 'center', 'justify-content': 'center',
            'background-color': 'white', 'margin-top': '20px', 'border-radius': '10px', 'margin-right': '20px', 'margin-left':'20px', 'font-size':'12px', 'color':'green'}) ]
    else:
        return ['Engagements ',html.Br(), html.B("{:,.0f}".format(total_engagements), style= {'color':'black', 'font-size':'20px', 'margin-top':'40px', 'font-family':'Arial Black' }), html.Br(),  html.P('No change', style= {'width':'100px','display': 'flex', 'align-items': 'center', 'justify-content': 'center',
            'background-color': 'lightgrey', 'margin-top': '20px', 'border-radius': '10px', 'margin-right': '20px', 'margin-left':'20px', 'font-size':'12px',  'color':'green',}) ]
    
@app.callback(
    Output('impressions', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)    
def update_impressions(start_date, end_date):
    filtered_impressions = engagements[(engagements['Date'] >= start_date) & (engagements['Date'] <= end_date)]
    total_impressions = filtered_impressions['Impressions'].sum()
        # Calculate previous period start and end dates based on selected period
    selected_period_start = pd.to_datetime(start_date)
    selected_period_end = pd.to_datetime(end_date)
    previous_period_start = selected_period_start - (selected_period_end - selected_period_start)
    previous_period_end = selected_period_start - timedelta(days=1)  # End 1 day before the selected period start
    
    # Filter engagements for previous period
    previous_period_impressions = engagements[(engagements['Date'] >= previous_period_start) & (engagements['Date'] <= previous_period_end)]
    previous_period_total_impressions = previous_period_impressions['Impressions'].sum()
    
    percentage_change = calculate_percentage_change(total_impressions, previous_period_total_impressions)
    if percentage_change < 0:
        return ['Impressions ',html.Br(), html.B("{:,.0f}".format(total_impressions), style= {'color':'black', 'font-size':'20px','font-family':'Arial Black' }), html.Br(),  html.P(f"{percentage_change}%", style= {'width':'100px','display': 'flex', 'align-items': 'center', 'justify-content': 'center',
                  'background-color': 'white', 'margin-top': '20px', 'border-radius': '10px', 'margin-right': '20px', 'margin-left':'20px', 'font-size':'12px', 'color':'red',}) ]
    elif percentage_change > 0:
        return ['Impressions ',html.Br(), html.B("{:,.0f}".format(total_impressions), style= {'color':'black', 'font-size':'20px','font-family':'Arial Black' }), html.Br(),  html.P(f"{percentage_change}%", style= {'width':'100px','display': 'flex', 'align-items': 'center', 'justify-content': 'center',
            'background-color': 'white', 'margin-top': '20px', 'border-radius': '10px', 'margin-right': '20px', 'margin-left':'20px', 'font-size':'12px', 'color':'green'}) ]
    else:
        return ['Impressions ',html.Br(), html.B("{:,.0f}".format(total_impressions), style= {'color':'black', 'font-size':'20px', 'margin-top':'40px', 'font-family':'Arial Black' }), html.Br(),  html.P('No change', style= {'width':'100px','display': 'flex', 'align-items': 'center', 'justify-content': 'center',
            'background-color': 'lightgray', 'margin-top': '20px', 'border-radius': '10px', 'margin-right': '20px', 'margin-left':'20px', 'font-size':'12px',  'color':'green',}) ]
    
@app.callback(
    Output('engagement_rate', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
) 
def update_engagements_rate(start_date, end_date):
    filtered_impressions = engagements[(engagements['Date'] >= start_date) & (engagements['Date'] <= end_date)]
    total_impressions = filtered_impressions['Impressions'].sum()
    total_engagements = filtered_impressions['Engagements'].sum()
    engagement_rate = (total_engagements / total_impressions)*100
    return ['Engagement Rate', html.Br(), html.B("{:,.2f}%".format(engagement_rate), style= {'color':'black', 'font-size':'20px', 'margin-top':'40px', 'font-family':'Arial Black' })]

@app.callback(
    Output('emv_value', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)   
def update_emv(start_date, end_date):
    filtered_impressions = engagements[(engagements['Date'] >= start_date) & (engagements['Date'] <= end_date)]
    total_impressions = filtered_impressions['Impressions'].sum()
    total_engagements = filtered_impressions['Engagements'].sum()
    engagement_rate = total_engagements / total_impressions
    emv = engagement_rate*total_impressions*6.59
    return ['Earned Media Value', html.Br(), html.B("${:,.0f}".format(emv), style= {'color':'black', 'font-size':'20px', 'margin-top':'40px', 'font-family':'Arial Black' })]
    

@app.callback(
    Output('engagements-graph', 'figure'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('variable-dropdown', 'value')]
)
def update_graph(start_date, end_date, selected_variable):
    filtered_data = engagements[(engagements['Date'] >= start_date) & (engagements['Date'] <= end_date)]
    fig = px.line(filtered_data, x='Date', y=selected_variable, color_discrete_sequence=['#b51a00'])
    return fig

@app.callback(
    Output('optimal-graph', 'figure'),
    Output('best-day', 'children'),
    Output('optimal-days', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('variable-dropdown', 'value')]
)
def update_graph(start_date, end_date, selected_variable):
    filtered_data = engagements[(engagements['Date'] >= start_date) & (engagements['Date'] <= end_date)]
    filtered_data['DayOfWeek'] = filtered_data['Date'].dt.day_name()
    engagements_by_day = filtered_data.groupby('DayOfWeek')[['Engagements','Impressions']].sum().reset_index()
    engagements_by_day['EngagementRate'] = (engagements_by_day['Engagements'] / engagements_by_day['Impressions']) * 100
    engagements_by_day['EngagementRate'] = engagements_by_day['EngagementRate'].map("{:.2f}".format)
    if selected_variable == 'Engagements':
        engagements_by_day = engagements_by_day.sort_values(by='Engagements', ascending=False).reset_index(drop=True)
        optimal_day = engagements_by_day.loc[0, 'DayOfWeek']
        fig = px.pie(engagements_by_day, names='DayOfWeek', values=selected_variable, color_discrete_sequence=['#b51a00']).update_layout(plot_bgcolor='white')
        return (fig, f"Engagement", ["With a total of ", html.B(engagements_by_day.loc[0, 'Engagements']), " reactions, the best day to post on LinkedIn to maximize your engagement is ", html.B(optimal_day), ". The second best day is ", html.B(engagements_by_day.loc[1, 'DayOfWeek'])])
    elif selected_variable == 'Impressions':
        engagements_by_day = engagements_by_day.sort_values(by='Impressions', ascending=False).reset_index(drop=True)
        optimal_day = engagements_by_day.loc[0, 'DayOfWeek']
        fig = px.histogram(engagements_by_day, x='DayOfWeek', y=selected_variable, color_discrete_sequence=['#b51a00']).update_layout(plot_bgcolor='white')
        return (fig, f"Reach", ["The best day to maximize the reach of your LinkedIn posts is ", html.B(optimal_day), " with total of ", html.B(engagements_by_day.loc[0, 'Impressions']), " impressions. The second best day is", html.B(engagements_by_day.loc[1, 'DayOfWeek']), "."])
    else:
        engagements_by_day = engagements_by_day.sort_values(by='EngagementRate', ascending=False).reset_index(drop=True)
        optimal_day = engagements_by_day.loc[0, 'DayOfWeek']
        fig = px.histogram(engagements_by_day, x='DayOfWeek', y=selected_variable, color_discrete_sequence=['#b51a00']).update_layout(plot_bgcolor='white')
        return (fig, f"Engagement rate", [ "Increase your engagement rate on your LinkedIn posts by posting on ", html.B(optimal_day), " with (", html.B(engagements_by_day.loc[0, 'EngagementRate']), "%) engagement rate."])

@app.callback(
    Output('audience-graph', 'figure'),
    [Input('audience-dropdown', 'value')]
)
def update_audience_graph(selected_variable):
    fig = px.bar(demographics[demographics['Top Demographics'] == selected_variable], x='Percentage', y='Value', orientation='h', title=f'Top audience by {selected_variable}', labels={'Percentage': 'Percentage (%)', 'Value': 'Top Values'},color_discrete_sequence=['#b51a00']).update_layout(plot_bgcolor='white')
    return fig

@app.callback(
    Output('top-posts-preview', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('posts-dropdown', 'value')]
)
def update_top_posts(start_date, end_date, selected_variable):
    filtered_data = top_posts_preview[(top_posts_preview['Post publish date'] >= start_date) & (top_posts_preview['Post publish date'] <= end_date)]
    if selected_variable == 'Top engaging posts':
        data = filtered_data.sort_values(by='Engagements', ascending=False).reset_index(drop=True).head()
        return dbc.Table(id = 'top-posts-preview',
            children=[
                html.Tr([
                    html.Th("Published on"),
                    html.Th("Posts"),
                    html.Th(""),
                    html.Th("Impressions"),
                    html.Th("Reactions"),
                    html.Th("Post Link"),
                    html.Th("Topics"),
                ], style={'background-color': 'lightgray'}),  # Header row styling
            ] +
            # Rows
            [html.Tr([
                html.Td(row['Post publish date'], style={'width':'200px', 'margin-left': '10px'}),
                html.Td(html.Img(src=row['Thumbnail'], height=100) if row['Thumbnail'] else html.P('Thumbnail not found.')),
                html.Td(row['Description'],style={'width':'800px', 'margin-left': '10px', 'margin-right': '40px'}),
                html.Td(html.B(row['Impressions']), style={'width':'200px', 'margin-left': '20px'}),
                html.Td(html.B(row['Engagements']),style={'width':'200px',}),
                html.Td(html.A("View on LI", href=row['Post URL']), style={'width':'200px',}),
                html.Td(row['Topics'],style={'width':'300px',}),
            ], style={'border-bottom': '1px solid gray', 'padding': '10px','width':'1700px'}) for _, row in data.iterrows()],
            bordered=True,
            responsive=True,
            striped=True,
            hover=True,
            style={'margin': '20px', 'width': '1700px'}  # Overall table styling
        )
    else:
        data = filtered_data.sort_values(by='Impressions', ascending=False).reset_index(drop=True).head(10)
        return dbc.Table(id = 'top-posts-preview',
            children=[
                html.Tr([
                    html.Th("Published on"),
                    html.Th("Posts"),
                    html.Th(""),
                    html.Th("Impressions"),
                    html.Th("Reactions"),
                    html.Th("Post Link"),
                    html.Th("Topics"),
                ], style={'background-color': 'lightgray'}),  # Header row styling
            ] +
            # Rows
            [html.Tr([
                html.Td(row['Post publish date'], style={'width':'200px', 'margin-left': '10px'}),
                html.Td(html.Img(src=row['Thumbnail'], height=100) if row['Thumbnail'] else html.P('Thumbnail not found.')),
                html.Td(row['Description'],style={'width':'800px', 'margin-left': '10px'}),
                html.Td(html.B(row['Impressions']), style={'width':'200px', 'margin-left': '20px'}),
                html.Td(html.B(row['Engagements']),style={'width':'200px',}),
                html.Td(html.A("View on LI", href=row['Post URL']), style={'width':'200px',}),
                html.Td(row['Topics'],style={'width':'300px',}),
            ], style={'border-bottom': '1px solid gray', 'padding': '10px','width':'1700px'}) for _, row in data.iterrows()],
            bordered=True,
            responsive=True,
            striped=True,
            hover=True,
            style={'margin': '20px', 'width': '100%'}  # Overall table styling
        )

if __name__ == '__main__':
    app.run_server(debug=True)
